In [3]:
import os
import fnmatch

def load_gitignore_patterns(gitignore_path):
    """读取 .gitignore 文件并生成忽略模式列表"""
    patterns = []
    try:
        with open(gitignore_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):  # 忽略注释行
                    patterns.append(line)
    except FileNotFoundError:
        print(f"未找到 {gitignore_path} 文件，所有文件将被读取。")
    return patterns

def is_ignored(file_path, patterns):
    """检查文件是否匹配 .gitignore 中的任何模式"""
    for pattern in patterns:
        if fnmatch.fnmatch(file_path, pattern):
            return True
    return False

def read_files_in_folder(folder_path, gitignore_path=".gitignore"):
    patterns = load_gitignore_patterns(gitignore_path)
    prompt_text = ""
    for root, _, files in os.walk(folder_path):
        # ignore folder components and containers
        if 'components' in root or 'containers' in root:
            continue
        for file_name in files:
            file_path = os.path.join(root, file_name)
            # 跳过符合 .gitignore 模式的文件
            relative_path = os.path.relpath(file_path, start=folder_path)
            if is_ignored(relative_path, patterns):
                print(f"忽略文件: {file_path}")
                continue
            # ignore .css, .png, .svg file
            if file_name.endswith('.css') or file_name.endswith('.png') or file_name.endswith('.svg'):
                continue
            # ignore some specific file
            if file_name in ['package-lock.json', 'package.11json']:
                continue
            
            # 读取文件内容
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    file_content = file.read()
                # 组合文件路径和内容
                prompt_text += f"路径: {file_path}\n代码内容:\n{file_content}\n\n"
            except Exception as e:
                print(f"无法读取文件 {file_path}: {e}")
    return prompt_text

# 设置文件夹路径
path = ["../frontend/", "../backend/"]
# folder_path = "../frontend/"
# folder_path = "../backend/"
for folder_path in path:
    prompt_text = read_files_in_folder(folder_path)

    # 将结果写入一个输出文件，或者直接打印
    output_file_path = f"prompt_{folder_path.replace('..', '').replace('/', '')}.txt"
    with open(output_file_path, "w", encoding="utf-8") as output_file:
        output_file.write(prompt_text)

    print(f"已生成 prompt 文本，可在 {output_file_path} 中查看。")

忽略文件: ../frontend/.DS_Store
无法读取文件 ../frontend/node_modules/.DS_Store: 'utf-8' codec can't decode byte 0xc6 in position 334: invalid continuation byte
无法读取文件 ../frontend/node_modules/.cache/default-development/index.pack: 'utf-8' codec can't decode byte 0xf7 in position 12: invalid start byte
无法读取文件 ../frontend/node_modules/.cache/default-development/0.pack: 'utf-8' codec can't decode byte 0xcc in position 9: invalid continuation byte
无法读取文件 ../frontend/src/.DS_Store: 'utf-8' codec can't decode byte 0x80 in position 1114: invalid start byte
已生成 prompt 文本，可在 prompt_frontend.txt 中查看。
忽略文件: ../backend/.DS_Store
忽略文件: ../backend/django_debug.log
无法读取文件 ../backend/users/.DS_Store: 'utf-8' codec can't decode byte 0xc6 in position 610: invalid continuation byte
无法读取文件 ../backend/users/migrations/__pycache__/__init__.cpython-39.pyc: 'utf-8' codec can't decode byte 0x93 in position 8: invalid start byte
无法读取文件 ../backend/users/migrations/__pycache__/0001_initial.cpython-311.pyc: 'utf-8' codec c

In [4]:
import yfinance as yf
from datetime import datetime


asset ='AAPL'
period="1mo"
interval="1d"
stock = yf.Ticker(asset)
history_data = stock.history(period=period, interval=interval)

if not history_data.empty:
    print(f"API Returns for {asset}:")
    print(history_data)
    print("\n")
    
    for date, row in history_data.iterrows():
        print(f"Date: {date.date()}")
        print(f"Open Price: {row['Open']}")
        print(f"Close Price: {row['Close']}")
        print(f"High Price: {row['High']}")
        print(f"Low Price: {row['Low']}")
        print(f"Volume: {row['Volume']}")
        print(f"Change (24h): {(row['Close'] - row['Open']) / row['Open'] * 100}%")
        print(f"Market Cap: {stock.info.get('marketCap', 0)}")
        print("\n")
else:
    print(f"No historical price data available for {asset}")


API Returns for AAPL:
                                 Open        High         Low       Close  \
Date                                                                        
2024-10-23 00:00:00-04:00  233.822752  234.881585  227.509690  230.506393   
2024-10-24 00:00:00-04:00  229.727257  230.566345  228.158990  230.316620   
2024-10-25 00:00:00-04:00  229.487523  232.963694  229.317711  231.155685   
2024-10-28 00:00:00-04:00  233.063595  234.472034  232.294437  233.143494   
2024-10-29 00:00:00-04:00  232.843827  234.072471  232.064686  233.413193   
2024-10-30 00:00:00-04:00  232.354358  233.213414  229.297724  229.847122   
2024-10-31 00:00:00-04:00  229.087951  229.577418  225.122313  225.661728   
2024-11-01 00:00:00-04:00  220.727153  225.102345  220.027926  222.665024   
2024-11-04 00:00:00-05:00  220.747138  222.545148  219.468546  221.766006   
2024-11-05 00:00:00-05:00  221.556241  223.703872  220.896963  223.204422   
2024-11-06 00:00:00-05:00  222.365355  225.821559  220

In [8]:
ticker = yf.Ticker("TSLA")

ticker.info

$TSLFASFA: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")


No historical price data available for AAPL


In [13]:
from binance.client import Client
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize Binance Client
api_key = os.getenv("BINANCE_API_KEY")
api_secret = os.getenv("BINANCE_API_SECRET")
client = Client(api_key=api_key, api_secret=api_secret)

def test_current_data(symbol):
    """
    Test fetching the current price and market data for a given cryptocurrency symbol.
    :param symbol: Example 'BTCUSDT'
    """
    try:
        ticker = client.get_ticker(symbol=symbol)
        weighted_avg_price = float(ticker['weightedAvgPrice'])
        volume = float(ticker['volume'])
        market_cap = weighted_avg_price * volume  # Calculate market capitalization

        current_data = {
            'price': float(ticker['lastPrice']),
            'open': float(ticker['openPrice']),
            'high': float(ticker['highPrice']),
            'low': float(ticker['lowPrice']),
            'volume': float(ticker['volume']),
            'market_cap': market_cap,
            'date': datetime.utcfromtimestamp(int(ticker['closeTime']) / 1000).date()
            
        }
        print("Current Data:", current_data)
    except Exception as e:
        print(f"Error fetching current data for {symbol}: {e}")

def test_historical_data(symbol, interval="1d", start_date=None, end_date=None):
    """
    Test fetching historical price data for a given cryptocurrency symbol.
    :param symbol: Example 'BTCUSDT'
    :param interval: Time interval (e.g., '1m', '15m', '1h', '4h', '1d')
    :param start_date: Start date in 'YYYY-MM-DD' format
    :param end_date: End date in 'YYYY-MM-DD' format
    """
    try:
        if start_date is None:
            start_date = (datetime.utcnow() - timedelta(days=30)).strftime('%Y-%m-%d')
        if end_date is None:
            end_date = datetime.utcnow().strftime('%Y-%m-%d')

        klines = client.get_historical_klines(
            symbol,
            interval,
            start_str=start_date,
            end_str=end_date
        )

        historical_data = []
        for kline in klines:
            historical_data.append({
                'date': datetime.utcfromtimestamp(kline[0] / 1000).date(),
                'open': float(kline[1]),
                'high': float(kline[2]),
                'low': float(kline[3]),
                'close': float(kline[4]),
                'volume': float(kline[5]),
            })

        print("Historical Data:")
        for entry in historical_data:  # Print the first 5 records for brevity
            print(entry)
    except Exception as e:
        print(f"Error fetching historical data for {symbol}: {e}")

# Test the functions
test_symbol = "BTCUSDT"  # Replace with the trading pair you want to test
test_current_data(test_symbol)
test_historical_data(test_symbol, interval="1d", start_date="2024-10-01", end_date="2024-11-23")


Current Data: {'price': 98596.01, 'open': 98680.97, 'high': 99588.01, 'low': 97122.11, 'volume': 42579.34984, 'market_cap': 4199791771.0638766, 'date': datetime.date(2024, 11, 23)}
Historical Data:
{'date': datetime.date(2024, 10, 1), 'open': 63327.6, 'high': 64130.63, 'low': 60164.0, 'close': 60805.78, 'volume': 43671.48108}
{'date': datetime.date(2024, 10, 2), 'open': 60804.92, 'high': 62390.31, 'low': 60000.0, 'close': 60649.28, 'volume': 31534.70118}
{'date': datetime.date(2024, 10, 3), 'open': 60649.27, 'high': 61477.19, 'low': 59828.11, 'close': 60752.71, 'volume': 26221.43472}
{'date': datetime.date(2024, 10, 4), 'open': 60752.72, 'high': 62484.85, 'low': 60459.9, 'close': 62086.0, 'volume': 21294.65994}
{'date': datetime.date(2024, 10, 5), 'open': 62086.0, 'high': 62370.56, 'low': 61689.26, 'close': 62058.0, 'volume': 7807.46141}
{'date': datetime.date(2024, 10, 6), 'open': 62058.01, 'high': 62975.0, 'low': 61798.97, 'close': 62819.91, 'volume': 8906.86177}
{'date': datetime.da

In [19]:
from pycoingecko import CoinGeckoAPI

# Initialize CoinGecko client
cg = CoinGeckoAPI()

def get_market_cap(crypto_id):
    """
    Fetch the market cap of a cryptocurrency using CoinGecko API.
    :param crypto_id: The CoinGecko ID of the cryptocurrency (e.g., 'bitcoin' for BTC)
    :return: Market cap in USD
    """
    try:
        data = cg.get_coin_market_chart_by_id(id=crypto_id, vs_currency='usd', days=1)
        market_cap = data['market_caps'][0][1]  # Get the most recent market cap value
        return market_cap
    except Exception as e:
        print(f"Error fetching market cap for {crypto_id}: {e}")
        return None

# Example usage
btc_market_cap = get_market_cap('bitcoin')
print("Bitcoin Market Cap:", btc_market_cap)


Bitcoin Market Cap: 1957819789649.4329


In [2]:
import yfinance as yf

stock = yf.Ticker('AAPL')
print(stock.info)

{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '(408) 996-1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as w

In [ ]:
# testing the market rank
import requests

# Replace with your actual API endpoint
BASE_URL = "http://localhost:8000/api/market/sector"
SECTOR = "technology"  # Change this to test different sectors

def test_sector_top_stocks():
    url = f"{BASE_URL}/{SECTOR}/"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print("API Test Successful!")
            print("Response Data:")
            for stock in response.json():
                print(
                    f"Symbol: {stock['symbol']}, Price: {stock['price']}, "
                    f"24h Change: {stock['change_24h']}%, Market Cap: {stock['market_cap']}"
                )
        else:
            print(f"API Test Failed! Status Code: {response.status_code}")
            print("Response:", response.json())
    except Exception as e:
        print(f"An error occurred while testing the API: {e}")

if __name__ == "__main__":
    test_sector_top_stocks()


In [18]:
import requests
from bs4 import BeautifulSoup

# URL for Yahoo Finance Markets
url = "https://finance.yahoo.com/markets/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
}

# Fetch the webpage
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Locate the market overview section
carousel = soup.find("div", {"class": "scroll-carousel"})
regions = carousel.find_all("section", {"data-testid": "world-indices"})

# Parse each region
for region in regions:
    region_name = region.find("h3").text.strip()
    print(f"Region: {region_name}")
    
    table = region.find("table", {"class": "markets-table"})
    rows = table.find("tbody").find_all("tr")
    
    for row in rows:
        symbol = row.find("a", {"data-testid": "table-cell-ticker"}).text.strip()
        price = row.find("fin-streamer", {"data-field": "regularMarketPrice"}).text.strip()
        change = row.find("fin-streamer", {"data-field": "regularMarketChangePercent"}).text.strip()
        print(f"  {symbol} - Price: {price}, Change: {change}")
    print()


Region: Americas
  S&P 500 - Price: 5,987.37, Change: (+0.30%)
  Dow 30 - Price: 44,736.57, Change: (+0.99%)
  Nasdaq - Price: 19,054.84, Change: (+0.27%)
  Russell 2000 - Price: 2,442.03, Change: (+1.47%)
  VIX - Price: 14.75, Change: (+1.03%)
  IBOVESPA - Price: 129,036.10, Change: (-0.07%)
  S&P/TSX Composite index - Price: 25,410.35, Change: (-0.13%)
  US Dollar Index - Price: 106.79, Change: (-0.03%)

Region: Europe
  FTSE 100 - Price: 8,263.53, Change: (-0.34%)
  CAC 40 - Price: 7,214.15, Change: (-0.60%)
  DAX                           P - Price: 19,304.25, Change: (-0.52%)
  Euronext 100 Index - Price: 1,432.16, Change: (-0.72%)
  EURO STOXX 50                 I - Price: 4,768.27, Change: (-0.66%)
  MSCI EUROPE - Price: 2,031.16, Change: (-0.14%)
  British Pound Index - Price: 125.68, Change: (+0.28%)
  Euro Index - Price: 104.96, Change: (+0.76%)

Region: Asia
  SSE Composite Index - Price: 3,259.76, Change: (-0.12%)
  Nikkei 225 - Price: 38,442.00, Change: (-0.87%)
  Hang Sen

In [51]:
import requests
import os

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

url = "https://api.x.ai/v1/chat/completions"

XAI_API_KEY = os.getenv("XAI_API_KEY")

print(XAI_API_KEY)  

# 定义请求数据
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {XAI_API_KEY}"
}

payload = {
    "messages": [
        {
            "role": "system",
            "content": "You are Grok, a chatbot inspired by the Hitchhikers Guide to the Galaxy."
        },
        {
            "role": "user",
            "content": "What is the meaning of life, the universe, and everything?"
        }
    ],
    "model": "grok-beta",
    "stream": False,
    "temperature": 0
}

# 发送 POST 请求
response = requests.post(url, json=payload, headers=headers)

# 检查响应
if response.status_code == 200:
    print("Response:", response.json())
else:
    print("Error:", response.status_code, response.text)


xai-bhGS91zRWYxFVbb3N99O99pdMTY3EHaDh9wuGA30krmetRHWTUJZL4Rla4WRZWAE3bEzmSAjb2btDlCP
Response: {'id': 'f9d0b395-bd5c-411a-b093-b8260d1aace2', 'object': 'chat.completion', 'created': 1732693222, 'model': 'grok-beta', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Ah, the ultimate question! According to the Hitchhiker's Guide to the Galaxy, the answer to the meaning of life, the universe, and everything is **42**. However, the Guide also points out that the question itself might be more important than the answer. So, while 42 is the answer, what exactly the question means is still up for debate. \n\nWould you like to explore what kind of question might fit this answer, or perhaps discuss the philosophical implications of this numeric response?", 'refusal': None}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 41, 'completion_tokens': 103, 'total_tokens': 144, 'prompt_tokens_details': {'text_tokens': 41, 'audio_tokens': 0, 'image_tokens': 0, 'cached_tokens': 

In [2]:
import requests
from bs4 import BeautifulSoup

# Yahoo Finance URL
YAHOO_FINANCE_URL = "https://finance.yahoo.com/markets/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
}

def fetch_page_content(url):
    """Fetch and parse the page content from Yahoo Finance."""
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data from Yahoo Finance. Status code: {response.status_code}")
    return BeautifulSoup(response.text, "html.parser")

def fetch_tops(soup, section_id):
    """Extract top gainers or top losers."""
    stocks = []
    # Find the section by data-id
    section = soup.find("li", {"data-id": section_id})
    if not section:
        print(f"Section with id {section_id} not found.")
        return stocks

    # Find all rows in the section
    rows = section.find_all("li", class_="dock-item primary font-default yf-46ugf5 clickability hover hasSymbolOptions")
    if not rows:
        print(f"No rows found in the section {section_id}.")
        return stocks

    # Extract data from each row
    for row in rows:
        try:
            symbol = row.find("span", class_="symbol").text.strip()
            name = row.find("span", class_="longName")["title"].strip()
            price = float(row.find("fin-streamer", {"data-field": "regularMarketPrice"})["data-value"])
            change_percent = float(row.find("fin-streamer", {"data-field": "regularMarketChangePercent"})["data-value"])
            stocks.append({
                "symbol": symbol,
                "name": name,
                "price": price,
                "change_percent": f"{change_percent:.2f}%"
            })
        except Exception as e:
            print(f"Error parsing row: {e}")
    return stocks

# Main Test
try:
    print("Fetching Yahoo Finance page content...")
    soup = fetch_page_content(YAHOO_FINANCE_URL)
    
    print("Fetching Top Gainers...")
    top_gainers = fetch_tops(soup, "topGainers")
    print(f"Top Gainers: {top_gainers}")
    
    print("Fetching Top Losers...")
    top_losers = fetch_tops(soup, "topLosers")
    print(f"Top Losers: {top_losers}")

except Exception as e:
    print(f"Error: {e}")


Fetching Yahoo Finance page content...
Fetching Top Gainers...
Top Gainers: [{'symbol': 'IREN', 'name': 'Iris Energy Limited', 'price': 12.4, 'change_percent': '29.71%'}, {'symbol': 'ARWR', 'name': 'Arrowhead Pharmaceuticals, Inc.', 'price': 26.15, 'change_percent': '24.23%'}, {'symbol': 'URBN', 'name': 'Urban Outfitters, Inc.', 'price': 47.49, 'change_percent': '18.31%'}, {'symbol': 'HUT', 'name': 'Hut 8 Corp.', 'price': 28.45, 'change_percent': '16.50%'}, {'symbol': 'CIFR', 'name': 'Cipher Mining Inc.', 'price': 6.45, 'change_percent': '14.56%'}]
Fetching Top Losers...
Top Losers: [{'symbol': 'SYM', 'name': 'Symbotic Inc.', 'price': 23.995, 'change_percent': '-35.86%'}, {'symbol': 'CHYI', 'name': 'Chykingyoung Investment Development Holdings Inc.', 'price': 6.01, 'change_percent': '-17.67%'}, {'symbol': 'DELL', 'name': 'Dell Technologies Inc.', 'price': 124.38, 'change_percent': '-12.25%'}, {'symbol': 'HPQ', 'name': 'HP Inc.', 'price': 34.66, 'change_percent': '-11.36%'}, {'symbol': 